In [24]:
import matplotlib.pyplot as plt
import pandas as pd
import subprocess

In [25]:
def find_import(test_file):
    try:
        return subprocess.check_output(["grep", "Require", test_file]).decode('utf-8').replace("Require Import ","").split(".\n")[:-1] #Select everything but the last '\n' that was split
    except subprocess.CalledProcessError as e:
        print('No dependencies')
        return []

In [26]:
find_file_locations = lambda : subprocess.check_output(['bash', '-c', "find . -type f -name '*.v'"]).decode('utf-8').split('\n')
remove_file_extension = lambda x : x[:-2]
remove_file_name = lambda x : "/".join(x.split("/")[:-1])
remove_file_path = lambda x : x.split("/")[-1]
path_to_logic = lambda x : x[2:].replace('/','.')
path_to_R = lambda x : x[2:] if '/' not in x[2:] else '"'+x[2:]+'"'

In [27]:
df = pd.DataFrame({
    'file_location' : find_file_locations()[:-1]
})

df['file'] = df['file_location'].apply(remove_file_path)
df['file_name'] = df['file'].apply(remove_file_extension)
df['file_path'] = df['file_location'].apply(remove_file_name)
df['logical_path'] = df['file_path'].apply(path_to_logic)
df['R_path'] = df['file_path'].apply(path_to_R)
df['dependencies']=df['file_location'].apply(find_import)
df['import_name'] = df['logical_path']+ '.' +df['file_name']
df['out_deg'] = df['dependencies'].apply(lambda x: len(x))

No dependencies


In [28]:
list_of = list()
for i in df['dependencies']:
    for j in i:
        list_of.append(j)
counter_of = lambda y : list_of.count(y)
df['in_deg'] = df['import_name'].apply(counter_of)


In [29]:
df.sort_values(['in_deg', 'out_deg'], inplace=True, ascending=(True, False))

In [30]:
df

,file_location,file,file_name,file_path,logical_path,R_path,dependencies,import_name,out_deg,in_deg
35,./examples/Class2Relational/Class2Relational.f...,Class2Relational.functional.old.v,Class2Relational.functional.old,./examples/Class2Relational,examples.Class2Relational,"""examples/Class2Relational""","[List, core.Model, example.ClassMetamodel, exa...",examples.Class2Relational.Class2Relational.fun...,20,0
45,./examples/Class2Relational/Class2Relational.f...,Class2Relational.functional.v,Class2Relational.functional,./examples/Class2Relational,examples.Class2Relational,"""examples/Class2Relational""","[List, core.Model, example.ClassMetamodel, exa...",examples.Class2Relational.Class2Relational.fun...,20,0
19,./examples/Class2RelationalMV/InformationPrese...,InformationPreservation.v,InformationPreservation,./examples/Class2RelationalMV,examples.Class2RelationalMV,"""examples/Class2RelationalMV""","[String, List, Multiset, ListSet, Omega, core....",examples.Class2RelationalMV.InformationPreserv...,13,0
36,./examples/Class2Relational/theorems/thm_unrea...,thm_unreachability_byInductionDef.v,thm_unreachability_byInductionDef,./examples/Class2Relational/theorems,examples.Class2Relational.theorems,"""examples/Class2Relational/theorems""","[Coq.Logic.Eqdep_dec, Coq.Arith.EqNat, List, O...",examples.Class2Relational.theorems.thm_unreach...,13,0
37,./examples/Class2Relational/theorems/thm_c2r_c...,thm_c2r_complete.v,thm_c2r_complete,./examples/Class2Relational/theorems,examples.Class2Relational.theorems,"""examples/Class2Relational/theorems""","[Coq.Logic.Eqdep_dec, Coq.Arith.EqNat, List, S...",examples.Class2Relational.theorems.thm_c2r_com...,13,0
38,./examples/Class2Relational/theorems/thm_table...,thm_table_name_definedness_general_simpl.v,thm_table_name_definedness_general_simpl,./examples/Class2Relational/theorems,examples.Class2Relational.theorems,"""examples/Class2Relational/theorems""","[Coq.Logic.Eqdep_dec, Coq.Arith.EqNat, List, F...",examples.Class2Relational.theorems.thm_table_n...,11,0
31,./examples/HSM2FSM/HSM.v,HSM.v,HSM,./examples/HSM2FSM,examples.HSM2FSM,"""examples/HSM2FSM""","[Bool, String, List, Multiset, ListSet, Omega,...",examples.HSM2FSM.HSM,10,0
33,./examples/HSM2FSM/HSM2FSM.v,HSM2FSM.v,HSM2FSM,./examples/HSM2FSM,examples.HSM2FSM,"""examples/HSM2FSM""","[Bool, String, List, Multiset, ListSet, Omega,...",examples.HSM2FSM.HSM2FSM,10,0
21,./examples/Class2RelationalMV/PersonTest.v,PersonTest.v,PersonTest,./examples/Class2RelationalMV,examples.Class2RelationalMV,"""examples/Class2RelationalMV""","[String, List, core.CoqTL, core.utils.tPrint, ...",examples.Class2RelationalMV.PersonTest,8,0
42,./examples/Class2Relational/PersonTest.v,PersonTest.v,PersonTest,./examples/Class2Relational,examples.Class2Relational,"""examples/Class2Relational""","[String, List, core.CoqTL, core.utils.tPrint, ...",examples.Class2Relational.PersonTest,8,0


In [31]:
for index, row in df.iterrows():
    subprocess.run(['coqc', '-R', row.R_path, row.logical_path, row.file_location])